### Лабораторная работа №2. Теория игр
Выполнили студенты: Петренко Людмила М33001, Кусайкина Елизавета М33001, Шалимов Иван М33021

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import json
import simplex

**Матричная игра** с нулевой суммой задается прямоугольной матрицей, которая содержит стратегии 1ого игрока в строках и стратегии 2ого игрока в столбцах. Такая матрица однозначно определяет игру. Каждый элемент a[i][j] матрицы представляет собой сумму выигрыша, уплачиваемую игроком 1 игроку 2, если 1ый выбирает стратегию, соответствующую i-й строке, а 2ой выбирает стратегию, соответствующую j-му столбцу.

**Чистые стратегии** - это возможные ходы игроков, выбираемые ими с вероятностью 1.

**Нижняя чистая цена игры (максимин)** - 

**Верхняя чистая цена игры (минимакс)** - 


**Смешанные стратегии** - 

#### Решение матричной игры
* Матрица игры может быть упрощена с помощью удаления **доминируемых стратегий**. Стратегия i игрока 1 **доминирует** стратегию j игрока 1 если для всех чистых стратегий игрока 2 его выигрыш при стратегии i больше или равен выигрышу при стратегии j. В таком случае стратегию j можно вычеркнуть, поскольку она точно не будет использована.

* После упрощения у каждого игрока может остаться по одной стратегии - в таком случае решение игры найдено.

* Чистые стратегии выбираются так, чтобы первый игрок максимизировал свой возможный выигрыш, а второй - минимизировал свой возможный проигрыш. Если для пары чистых стратегий минимакс и максимин совпадают, то эта пара стратегий - седловая точка игры, а максимин и минимакс - чистая цена игры. Данная игра является разрешимой в чистых стратегиях.

* 

Согласно теореме фон Неймана, изменение всех значений матрицы на одно и то же число повлияет только на цену игры, но не на ее решение.

In [107]:
class Game:
    def __init__(self):
        self.matrix: np.array()
        self.simplify_flag = 1
        self.gamer = 'a'

    def load_data_from_json(self, filename: str) -> None:
        with open(filename) as f:
            data = json.load(f)
        self.gamer = data["gamer"]
        self.matrix = [[
            data['a ' + str(i)]['b ' + str(j)] for j in range(1, len(data['a ' + str(i)]) + 1)]
                for i in range(1, len(data))]
        self.original_matrix = np.copy(self.matrix)
        while self.simplify_flag:
            self.simplify_flag = self.simplify_matrix()

    def simplify_matrix(self, simplify_flag = 0):
        # удаление доминируемых стратегий игрока a
        for i in range(len(self.matrix)):
            for k in range(len(self.matrix)):
                if k == i:
                    continue
                if k >= len(self.matrix) or i >= len(self.matrix):
                    break
                number = 0
                for j in range(len(self.matrix[i])):
                    if self.matrix[i][j] <= self.matrix[k][j]:
                        number += 1
                if number == len(self.matrix[i]):
                    self.matrix = self.matrix[:i] + self.matrix[i + 1:]
                    simplify_flag = 1
        
        # удаление доминируемых стратегий игрока b
        for k in range(len(self.matrix[0])):
            for j in range(len(self.matrix[0])):
                number = 0
                if k == j:
                    continue
                if k >= len(self.matrix[0]) or j >= len(self.matrix[0]):
                    break
                for i in range(len(self.matrix)):
                    if self.matrix[i][j] <= self.matrix[i][k]:
                        number += 1
                if number == len(self.matrix):
                    for i in range(len(self.matrix)):
                        self.matrix[i] = self.matrix[i][:j] + self.matrix[i][j + 1:]
                    simplify_flag = 1
        return simplify_flag
    
    def solve(self):
        result = None#self.check_for_clean_strategies()
        if result is not None:
            if result < 0:
                if self.gamer == 'a':
                    self.gamer = 'b'
                else:
                    self.gamer = 'a'
                result = -result
            print(f"Решение в чистых стратегиях: \nигрок {self.gamer} победил с выигрышем {result}")
            return self.gamer, result
        
        print("Стратегии, оптимальные по Парето: \n", self.optimal_pareto())
        # else:
        #     print("Решения в чистых стратегиях не существует")
        
    def optimal_pareto(self):
        return 'a'

    def check_for_clean_strategies(self):
        if len(self.matrix) == 1 and len(self.matrix[0]) == 1:
           return self.matrix[0][0]
        
        self.minimax = min([max([self.matrix[i][j] for i in range(len(self.matrix))]) for j in range(len(self.matrix[0]))])
        self.maximin = max([min([self.matrix[i][j] for j in range(len(self.matrix[i]))]) for i in range(len(self.matrix))])

        print(self.maximin, self.minimax)

        if self.maximin == self.maximin:
            return self.minimax

In [108]:
game = Game()
game.load_data_from_json("data.json")
game.solve()


Стратегии, оптимальные по Парето:
 a
